Interacting Proteins

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u
'''
import sys
sys.path.append('C:\\Users\\brittany henderson\\GitHub\\WhenMutationsDontMatter\\')
import plot_utils as p'''

"\nimport sys\nsys.path.append('C:\\Users\\brittany henderson\\GitHub\\WhenMutationsDontMatter\\')\nimport plot_utils as p"

In [4]:
gbm = cptac.Gbm()
endo = cptac.Endometrial()
h = cptac.Hnscc()
l = cptac.Luad()
o = cptac.Ovarian()
col = cptac.Colon()
b = cptac.Brca()

cptac warning: The GBM dataset is under publication embargo until March 01, 2021. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 1)


cptac warning: The LUAD dataset is under publication embargo until July 01, 2020. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 4)


In [67]:
gene = 'PTEN'

In [92]:
ip = u.get_interacting_proteins('PTEN')

# RAS-RAF-MEK-ERK MAPK Pathway
ip.append('GRB2')
ip.append('SHC1') # (SHC in paper)
ip.append('SOS1') # (SOS in the paper) *GEF for RAS "recruitment to the plasma membrane requires GRB2, PIP2, PA" 
ip.append('KRAS') # (RAS in paper) possibly add "RAS regulators such as PHLPP, SHP-2, and NF-2"
ip.append('RAF1') # (RAF-1 in paper)
ip.append('MAP2K1') #(MEK1 protein name in paper)
ip.append('MAP2K2 ') # (MEK2 in paper)
ip.append('MAPK3') # (ERK1 in paper) https://www.uniprot.org/uniprot/P27361
ip.append('MAPK1') # (ERK2 in paper)

#mTOR Pathway
ip.append('GAB1')
#ip.append('PI3K')
ip.append('AKT1') # (AKT in paper) *binds PIP3 (see info for isoform AKT1 at: https://www.uniprot.org/uniprot/P31749)
ip.append('PDK1') #activate AKT
ip.append('MTORC2') #activate AKT
ip.append('GSK3B') # (GSK-3B in paper)inhibit cyclin D (akt inhibits GSK3B)
ip.append('FOXO1') #(FOXO in paper)
ip.append('TSC2') # repress mTOR, AKT inhibits
ip.append('MTOR') # promote cyclin D
ip.append('BAD')
ip.append('CAS9')

ip.append('CCND1') # Cyclin D1 -> G1/S 


singles = set(ip)
ip = list(singles)
print(len(ip))
#ip

46


In [93]:
mut_type_gbm = gbm.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = gbm.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = ip)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type_gbm = mut_type_gbm[['Mutation']] 
merged = ip_df.join(mut_type_gbm) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()
del_wt

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 72 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: C3L-01834 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: The following columns were not found in the proteomics dataframe, so they were inserted into joined table, but filled with NaN: CAS9, MAP2K2 , MTORC2, PIK3R3 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 82 samples for the PTEN gene (C:\

Name,BAD_proteomics,AKT1_proteomics,PIK3CA_proteomics,PDGFRB_proteomics,PIK3CB_proteomics,PIK3R1_proteomics,PTEN_proteomics,KRAS_proteomics,NEDD4_proteomics,SOS1_proteomics,...,ROCK1_proteomics,PIK3C3_proteomics,PIK3CD_proteomics,GSK3B_proteomics,CSNK2A1_proteomics,CCND1_proteomics,MDM2_proteomics,USP7_proteomics,CAS9_proteomics,Mutation
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00104,0.216721,-0.065930,0.124542,-0.108555,-0.550558,-0.379646,-0.431991,-0.023009,-0.437514,0.086085,...,-0.319701,0.081642,-0.315003,0.015129,0.221578,-0.135982,-0.886531,-0.111215,NaN,Deletion
C3L-00365,0.086908,-0.285929,-0.211025,-0.109383,0.191167,-0.283144,-0.265044,0.088499,0.179955,0.198497,...,0.013908,-0.027524,-0.629943,-0.200441,0.169391,NaN,NaN,-0.064326,NaN,Deletion
C3L-00674,-0.400225,-0.222607,-0.573919,0.414121,-0.548042,0.061201,0.390190,-0.071808,0.265735,-0.166658,...,-0.048311,0.062739,0.085276,0.011714,-0.208625,-0.092478,NaN,-0.100264,NaN,Deletion
C3L-00677,0.618380,-0.370584,-0.360962,-0.141799,-0.397225,-0.496340,-0.915526,-0.438601,0.706601,-0.014030,...,-0.046639,0.087752,-0.382652,-0.228395,-0.292558,-0.454459,-0.564682,-0.110632,NaN,Deletion
C3L-01040,0.557178,0.004573,0.137414,0.812053,-0.395284,-0.146416,0.680291,0.108811,0.133808,-0.269299,...,0.237764,-0.057555,-0.537787,0.302245,0.132716,0.272387,-0.243835,0.088481,NaN,Deletion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-03183,-0.071163,-0.002002,-0.218186,0.121787,-0.223580,-0.093624,-0.725120,-0.605825,0.418850,-0.014041,...,0.141372,-0.092783,0.395350,-0.375038,-0.015279,0.873299,NaN,0.381688,NaN,Deletion
C3N-03184,-0.128968,0.501695,-0.049803,0.361701,-0.036881,-0.164426,-0.242603,-0.130796,0.028673,-0.132077,...,-0.158482,-0.160911,-0.556546,-0.134177,0.312023,NaN,NaN,0.313108,NaN,Deletion
C3N-03186,-0.179271,0.196700,0.815177,0.606401,0.236419,0.443719,0.428594,0.302745,0.013211,-0.052877,...,-0.222781,-0.029083,-0.009546,0.264166,-0.069012,NaN,0.137052,-0.057040,NaN,Deletion


In [95]:
cols = list(del_wt.columns[:-1])

# Get only sig sites
g_sig = u.wrap_ttest(del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if g_sig is not None:
    g_sig_list = list(g_sig.Comparison)
else: 
    l_sig_list = None
print('significant pvals: \n',g_sig)

# Get all pvals
g = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
g_pval = g.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Gbm_P_Value'})
#g_pval = g_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)
g_pval

significant pvals: 
           Comparison       P_Value
0    PTEN_proteomics  1.219523e-07
1   MAPK3_proteomics  1.020660e-06
2  PIK3CD_proteomics  1.030815e-03
3    GRB2_proteomics  1.089951e-03
4  PIK3C3_proteomics  1.225949e-03
5    USP7_proteomics  4.017792e-03
6  MAP2K1_proteomics  4.138456e-03
7  PIK3R1_proteomics  4.282296e-03


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:320: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:328: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:251: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:320: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:328: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corre

,Proteomics,Gbm_P_Value
0,PTEN_proteomics,1.219523e-07
1,MAPK3_proteomics,1.020660e-06
2,PIK3CD_proteomics,1.030815e-03
3,GRB2_proteomics,1.089951e-03
4,PIK3C3_proteomics,1.225949e-03
5,USP7_proteomics,4.017792e-03
6,MAP2K1_proteomics,4.138456e-03
7,PIK3R1_proteomics,4.282296e-03
8,MAST2_proteomics,1.260293e-02
9,INPP4B_proteomics,1.455578e-02


In [96]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

gbm_d = {}

for prot in ip:
    dif_gbm = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    gbm_d[prot+'_proteomics'] = dif_gbm
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif_gbm)

gbm_df = pd.DataFrame.from_dict(gbm_d, orient='index', columns=['Gbm_Median'])
gbm_df = gbm_df.reset_index().rename(columns={'index':'Proteomics'})
gbm_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


,Proteomics,Gbm_Median
0,BAD_proteomics,0.087843
1,AKT1_proteomics,-0.169565
2,PIK3CA_proteomics,0.105746
3,PDGFRB_proteomics,-0.028576
4,PIK3CB_proteomics,0.042720
5,PIK3R1_proteomics,0.188390
6,PTEN_proteomics,0.511726
7,KRAS_proteomics,0.186256
8,NEDD4_proteomics,-0.079598
9,SOS1_proteomics,0.034435


In [100]:
g_merged = g_pval.merge(gbm_df, on='Proteomics',how='inner')
g_merged

,Proteomics,Gbm_P_Value,Gbm_Median
0,PTEN_proteomics,1.219523e-07,0.511726
1,MAPK3_proteomics,1.020660e-06,0.665105
2,PIK3CD_proteomics,1.030815e-03,0.492099
3,GRB2_proteomics,1.089951e-03,0.312146
4,PIK3C3_proteomics,1.225949e-03,0.102518
5,USP7_proteomics,4.017792e-03,-0.154035
6,MAP2K1_proteomics,4.138456e-03,0.592389
7,PIK3R1_proteomics,4.282296e-03,0.188390
8,MAST2_proteomics,1.260293e-02,0.139061
9,INPP4B_proteomics,1.455578e-02,0.341829


Endo

In [101]:
#Endo ttest

mut_type_gbm = endo.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = endo.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = ip)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type_gbm = mut_type_gbm[['Mutation']] 
merged = ip_df.join(mut_type_gbm) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Nonsense_Mutation','Frame_Shift_Ins','Frame_Shift_Del']
get = merged['Mutation'].isin(compare)
trunc_wt = merged[get]
trunc_wt['Mutation'] = np.where(
                trunc_wt['Mutation'] == 'Wildtype_Tumor', 'Wildtype_Tumor', 'Truncation')
trunc_wt['Mutation'].value_counts()
trunc_wt

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 20 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: The following columns were not found in the proteomics dataframe, so they were inserted into joined table, but filled with NaN: CAS9, MAP2K2 , MDM2, MTORC2, PIK3R3 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 7)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 69 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 7)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value i

Name,BAD_proteomics,AKT1_proteomics,PIK3CA_proteomics,PDGFRB_proteomics,PIK3CB_proteomics,PIK3R1_proteomics,PTEN_proteomics,KRAS_proteomics,NEDD4_proteomics,SOS1_proteomics,...,ROCK1_proteomics,PIK3C3_proteomics,PIK3CD_proteomics,GSK3B_proteomics,CSNK2A1_proteomics,CCND1_proteomics,MDM2_proteomics,USP7_proteomics,CAS9_proteomics,Mutation
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,0.01830,-0.2550,-0.4500,-0.4940,-0.0761,0.481,-0.526,-0.0603,-0.8590,-0.3670,...,-0.367,-0.41600,-0.25200,-0.0652,0.01510,-0.2170,NaN,0.1570,NaN,Truncation
C3L-00032,-0.07960,-0.0722,-0.1780,0.0143,0.3620,0.266,-0.941,0.1040,-0.7620,-0.1120,...,0.180,-0.00815,0.45700,-0.1370,-0.28000,NaN,NaN,-0.0873,NaN,Truncation
C3L-00098,-0.41700,-0.6410,0.2870,-1.3400,0.3730,-0.612,-0.379,-0.2560,-0.0684,0.2140,...,-0.164,0.04510,0.00543,-0.2090,0.35000,-1.5200,NaN,0.2730,NaN,Wildtype_Tumor
C3L-00137,-0.30400,0.1430,-0.3170,-0.6530,-0.7650,-1.910,-1.010,-0.1560,-1.2500,-0.2220,...,-0.178,-0.59600,-1.05000,0.0837,0.14300,NaN,NaN,0.3490,NaN,Truncation
C3L-00139,0.08390,-0.7250,0.2820,-0.4280,-0.1750,-0.496,0.130,-0.3450,-0.9890,-0.0330,...,-0.222,-0.07510,-0.20400,0.3390,0.61300,NaN,NaN,0.1950,NaN,Wildtype_Tumor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-01520,-0.15900,0.0217,-0.4860,-0.7660,0.0470,0.218,-1.910,0.6620,-0.9740,-0.2790,...,-0.176,0.36700,-0.12900,0.0758,-0.21200,0.2370,NaN,0.1850,NaN,Truncation
C3N-01521,-0.00364,0.1720,0.0820,-1.1200,1.1700,0.117,-0.616,-0.4350,-0.4740,-0.2130,...,-0.327,-0.28300,0.20500,0.1760,-0.00398,0.1190,NaN,0.1080,NaN,Wildtype_Tumor
C3N-01537,0.08140,-0.2300,-0.0312,-0.7420,-0.0841,-0.333,-0.164,-0.1370,-0.8110,-0.1140,...,-0.174,0.18900,0.74600,-0.2280,-0.07130,-0.2270,NaN,-0.1300,NaN,Wildtype_Tumor


In [102]:
cols = list(trunc_wt.columns[:-1])

# Get only sig sites
e_sig = u.wrap_ttest(trunc_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if e_sig is not None:
    e_sig_list = list(e_sig.Comparison)
else: 
    e_sig_list = None
print('significant pvals: \n',e_sig)

# Get all pvals
e_pval = u.wrap_ttest(trunc_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
e_pval = e_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'En_P_Value'})
e_pval

significant pvals: 
            Comparison   P_Value
0     PTEN_proteomics  0.000640
1     TP53_proteomics  0.001246
2   PIK3CA_proteomics  0.001761
3  CSNK2A1_proteomics  0.002364


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:320: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:328: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:251: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:320: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:328: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corre

,Proteomics,En_P_Value
0,PTEN_proteomics,0.000640
1,TP53_proteomics,0.001246
2,PIK3CA_proteomics,0.001761
3,CSNK2A1_proteomics,0.002364
4,EGFR_proteomics,0.006128
5,SHC1_proteomics,0.018340
6,CSNK2A2_proteomics,0.035238
7,PIK3C3_proteomics,0.036592
8,PIK3CD_proteomics,0.045489
9,PREX2_proteomics,0.053078


In [103]:
t = trunc_wt[trunc_wt.Mutation == "Truncation"]
wt = trunc_wt[trunc_wt.Mutation == "Wildtype_Tumor"]
trunc_med = t.median()
wt_med = wt.median()

en_d = {}

for prot in ip:
    dif = wt_med[prot+'_proteomics'] - trunc_med[prot+'_proteomics']
    en_d[prot+'_proteomics'] = dif
    #print(wt_med[prot+'_proteomics'], '-', trunc_med[prot+'_proteomics'], '=',dif)

en_df = pd.DataFrame.from_dict(en_d, orient='index', columns=['En_Median'])
en_df = en_df.reset_index().rename(columns={'index':'Proteomics'})
en_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


,Proteomics,En_Median
0,BAD_proteomics,0.11250
1,AKT1_proteomics,0.05160
2,PIK3CA_proteomics,0.35300
3,PDGFRB_proteomics,-0.17100
4,PIK3CB_proteomics,-0.03370
5,PIK3R1_proteomics,-0.33990
6,PTEN_proteomics,0.74660
7,KRAS_proteomics,-0.19180
8,NEDD4_proteomics,0.04500
9,SOS1_proteomics,0.14100


In [104]:
e_merged = e_pval.merge(en_df, on='Proteomics',how='inner')
e_merged

,Proteomics,En_P_Value,En_Median
0,PTEN_proteomics,0.000640,0.74660
1,TP53_proteomics,0.001246,0.80260
2,PIK3CA_proteomics,0.001761,0.35300
3,CSNK2A1_proteomics,0.002364,0.10412
4,EGFR_proteomics,0.006128,-0.52600
5,SHC1_proteomics,0.018340,0.24980
6,CSNK2A2_proteomics,0.035238,0.15970
7,PIK3C3_proteomics,0.036592,0.13160
8,PIK3CD_proteomics,0.045489,0.18243
9,PREX2_proteomics,0.053078,-0.45800


Ovarian

In [33]:
#o = cptac.Ovarian()

In [105]:
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest
gene = 'PTEN'

mut_type_gbm = o.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = o.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = ip)
prot_and_mutations = o.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
prot_and_mutations = prot_and_mutations.loc[:,~prot_and_mutations.columns.duplicated()] # drop duplicated columns
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type_gbm = mut_type_gbm[['Mutation']] 
merged = ip_df.join(mut_type_gbm) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 98 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 01OV029, 02OV015 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: The following columns were not found in the proteomics dataframe, so they were inserted into joined table, but filled with NaN: CAS9, CCND1, MAP2K2 , MDM2, MTORC2, PIK3R3 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 8)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 99 samples fo

Wildtype_Tumor    46
Deletion          20
Name: Mutation, dtype: int64

In [106]:
cols = list(del_wt.columns[:-1])

# Get only sig sites
o_sig = u.wrap_ttest(del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if o_sig is not None:
    o_sig_list = list(o_sig.Comparison)
else: 
    o_sig_list = None
print('significant pvals: \n',o_sig)

# Get all pvals
o = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
o_pval = o.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Ov_P_Value'})
#o_pval = o_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)
#o_pval

significant pvals: 
         Comparison   P_Value
0  PTEN_proteomics  0.000909


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:320: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:328: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:251: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:320: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:328: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corre

In [107]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

ov_d = {}

for prot in ip:
    dif = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    ov_d[prot+'_proteomics'] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

o_df = pd.DataFrame.from_dict(ov_d, orient='index', columns=['Ov_Median'])
o_df = o_df.reset_index().rename(columns={'index':'Proteomics'})
#o_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [108]:
o_merged = o_pval.merge(o_df, on='Proteomics',how='inner')
o_merged

,Proteomics,Ov_P_Value,Ov_Median
0,PTEN_proteomics,0.000909,0.169972
1,PIK3R2_proteomics,0.005865,-0.240986
2,PIK3CA_proteomics,0.008335,-0.199296
3,PIK3CB_proteomics,0.050313,-0.276354
4,PREX2_proteomics,0.060296,-0.505300
5,PTK2_proteomics,0.080941,-0.229494
6,GRB2_proteomics,0.086428,-0.184243
7,MAGI3_proteomics,0.150601,-0.133108
8,MAST2_proteomics,0.164860,0.085809
9,MTOR_proteomics,0.213423,-0.064073


Breast

In [40]:
#b = cptac.Brca()

In [110]:
mut_type = b.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = b.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = ip)
prot_and_mutations = b.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
prot_and_mutations = prot_and_mutations.loc[:,~prot_and_mutations.columns.duplicated()] # drop duplicated columns
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type = mut_type[['Mutation']] 
merged = ip_df.join(mut_type) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 113 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: The following columns were not found in the proteomics dataframe, so they were inserted into joined table, but filled with NaN: CAS9, CSNK2A1, MAP2K2 , MDM2, MTORC2, PIK3R3 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 113 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)
cptac warning: Due to dropping the specified levels, dataframe now has 2 duplicated column headers. (C:\Users\brittany 

Wildtype_Tumor    80
Deletion          24
Name: Mutation, dtype: int64

In [111]:
cols = list(del_wt.columns[:-1])

# Get only sig sites
b_sig = u.wrap_ttest(del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if b_sig is not None:
    b_sig_list = list(b_sig.Comparison)
else: 
    b_sig_list = None
print('significant pvals: \n',b_sig)

# Get all pvals
b_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True)
b_pval = b_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Brca_P_Value'})
#b_pval = b_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)

#b_pval

significant pvals: 
         Comparison   P_Value
0  PTEN_proteomics  0.000004


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:320: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:328: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:251: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:147: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals <= alphacBonf
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:251: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


In [112]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

b_d = {}

for prot in ip:
    dif = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    b_d[prot+'_proteomics'] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

b_df = pd.DataFrame.from_dict(b_d, orient='index', columns=['Brca_Median'])
b_df = b_df.reset_index().rename(columns={'index':'Proteomics'})
#b_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [113]:
b_merged = b_pval.merge(b_df, on='Proteomics',how='inner')
b_merged

,Proteomics,Brca_P_Value,Brca_Median
0,PTEN_proteomics,0.000004,1.00800
1,SHC1_proteomics,0.014172,-0.42620
2,EGFR_proteomics,0.016614,-0.58645
3,GRB2_proteomics,0.061517,0.31470
4,TP53_proteomics,0.068427,-0.11100
5,GSK3B_proteomics,0.088200,-0.18225
6,RAF1_proteomics,0.131515,0.21150
7,CCND1_proteomics,0.151628,0.46115
8,PIK3R1_proteomics,0.164475,1.03615
9,MAST2_proteomics,0.213636,0.29705


Colon

In [45]:
#col = cptac.Colon()

In [114]:
mut_type = col.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = col.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = ip)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type = mut_type[['Mutation']] 
merged = ip_df.join(mut_type) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 100 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 05CO045 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: The following columns were not found in the proteomics dataframe, so they were inserted into joined table, but filled with NaN: CAS9, CCND1, FOXO1, MAGI3, MAP2K2 , MAST2, MDM2, MTORC2, PIK3R3, PREX2 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_N

Wildtype_Tumor    76
Deletion          25
Name: Mutation, dtype: int64

In [115]:
cols = list(del_wt.columns[:-1])

# Get only sig sites
c_sig = u.wrap_ttest(del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if c_sig is not None:
    c_sig_list = list(c_sig.Comparison)
else: 
    c_sig_list = None
print('significant pvals: \n',c_sig)

# Get all pvals
c_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
c_pval = c_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Colon_P_Value'})
#c_pval = c_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)

#c_pval

significant pvals: 
 None


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:320: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:328: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\statsmodels\stats\multitest.py:251: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


In [116]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

d = {}

for prot in ip:
    dif = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    d[prot+'_proteomics'] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

c_df = pd.DataFrame.from_dict(d, orient='index', columns=['Colon_Median'])
c_df = c_df.reset_index().rename(columns={'index':'Proteomics'})
#c_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [117]:
c_merged = c_pval.merge(c_df, on='Proteomics',how='inner')
c_merged

,Proteomics,Colon_P_Value,Colon_Median
0,MAPK3_proteomics,0.003390,-0.19490
1,PTEN_proteomics,0.012872,0.20400
2,SHC1_proteomics,0.030107,-0.11610
3,PDK1_proteomics,0.050997,0.07900
4,KRAS_proteomics,0.062750,0.10560
5,PIK3C3_proteomics,0.116419,0.07995
6,INPP4B_proteomics,0.118135,-0.08220
7,PIK3CD_proteomics,0.131200,0.07600
8,CSNK2A1_proteomics,0.144425,0.04845
9,MVP_proteomics,0.182782,-0.23690


Head and Neck

In [52]:
#h = cptac.Hnscc()

In [118]:
mut_type = h.get_genotype_all_vars(gene)

# merge cnv with genotype all mut type
prot_and_mutations = h.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = ip)
#prot_and_mutations = h.reduce_multiindex(prot_and_mutations, levels_to_drop = 1)
prot_and_mutations = prot_and_mutations[prot_and_mutations.Sample_Status == "Tumor"] # drop Normal samples
ip_df = prot_and_mutations.iloc[:,:-4] #drop mutation, location cols
mut_type = mut_type[['Mutation']] 
merged = ip_df.join(mut_type) # merge mutation col from function (includes cnv)

# Keep two values to compare
compare = ['Wildtype_Tumor','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
del_wt['Mutation'].value_counts()

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 109 samples for the PTEN gene (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: C3L-02617, C3N-02727 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\cptac\dataset.py, line 383)
cptac warning: The following columns were not found in the proteomics dataframe, so they were inserted into joined table, but filled with NaN: CAS9, MAP2K2 , MDM2, MTORC2, PIK3R3 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 5)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 176 samples for

Wildtype_Tumor    87
Deletion          18
Name: Mutation, dtype: int64

In [119]:
cols = list(del_wt.columns[:-1])

# Get only sig sites
h_sig = u.wrap_ttest(del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if h_sig is not None:
    h_sig_list = list(h_sig.Comparison)
else: 
    h_sig_list = None
print('significant pvals: \n',h_sig)

# Get all pvals

h_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
h_pval = h_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Hnscc_P_Value'})
#h_pval = h_pval.replace(to_replace = r'_proteomics$', value = '', regex = True)

#h_pval


significant pvals: 
           Comparison   P_Value
0    PTEN_proteomics  0.000023
1  INPP4B_proteomics  0.000676
2  PIK3CA_proteomics  0.001677


C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\brittany henderson\Anaconda3\e

In [120]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "Wildtype_Tumor"]
del_med = d.median()
wt_med = wt.median()

h_d = {}

for prot in ip:
    dif = wt_med[prot+'_proteomics'] - del_med[prot+'_proteomics']
    h_d[prot+'_proteomics'] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

h_df = pd.DataFrame.from_dict(h_d, orient='index', columns=['Hnscc_Median'])
h_df = h_df.reset_index().rename(columns={'index':'Proteomics'})
#h_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\lib\nanfunctions.py:1076: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [121]:
h_merged = h_pval.merge(h_df, on='Proteomics',how='inner')
#del_wt[['PREX2_proteomics','Mutation']].dropna() # in median, not in pval (only 1 value for deletion)
h_merged

,Proteomics,Hnscc_P_Value,Hnscc_Median
0,PTEN_proteomics,0.000023,0.318812
1,INPP4B_proteomics,0.000676,0.591596
2,PIK3CA_proteomics,0.001677,-0.163663
3,USP13_proteomics,0.005645,-0.217338
4,TSC2_proteomics,0.012038,0.081507
5,PIK3CD_proteomics,0.014016,0.238171
6,EGFR_proteomics,0.017719,-0.467227
7,KRAS_proteomics,0.025498,-0.101291
8,SLC9A3R1_proteomics,0.031385,-0.155484
9,TP53_proteomics,0.031801,-0.705560


Lung

In [57]:
#l = cptac.Luad()

cptac warning: The LUAD dataset is under publication embargo until July 01, 2020. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 1)


In [132]:
mut_type = l.get_genotype_all_vars(gene)
mut_type = mut_type.rename(columns={'PTEN':'cnv'})

# different code because no somatic mutation data for pten (can't join to somatic mutations)
omics = l.join_omics_to_omics(df1_name = 'CNV', df2_name='proteomics',genes1="PTEN", 
    genes2=ip)
omics = l.reduce_multiindex(omics, levels_to_drop = 1, flatten = True)
omics = omics.drop(columns='PTEN_CNV')

# Get only tumor samples
p = l.get_proteomics(tissue_type='tumor')
tumor_ids = list(p.index)
get = omics.index.isin(tumor_ids)
omics = omics[get]

merged = omics.join(mut_type) # checked and there is 110 tumor samples for lung

compare = ['No_Mutation','Deletion']
get = merged['Mutation'].isin(compare)
del_wt = merged[get]
print(del_wt['Mutation'].value_counts())
del_wt = del_wt.dropna(axis='columns', how='all')
del_wt = del_wt.loc[:,~del_wt.columns.duplicated()] # SH1, PRK3R1 both have double columns

No_Mutation    88
Deletion       17
Name: Mutation, dtype: int64


cptac warning: The following columns were not found in the proteomics dataframe, so they were inserted into joined table, but filled with NaN: CAS9, CSNK2A1, CSNK2A2, MAP2K2 , MDM2, MTOR, MTORC2, PIK3R3, RAF1, SOS1, TSC2 (C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py, line 6)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: C3L-00001.N, C3L-00009.N, C3L-00080.N, C3L-00083.N, C3L-00093.N, C3L-00094.N, C3L-00095.N, C3L-00140.N, C3L-00144.N, C3L-00263.N, C3L-00279.N, C3L-00368.N, C3L-00412.N, C3L-00422.N, C3L-00510.N, C3L-00604.N, C3L-00893.N, C3L-00913.N, C3L-00973.N, C3L-01330.N, C3L-01632.N, C3L-01682.N, C3L-01683.N, C3L-01889.N, C3L-01890, C3L-01890.N, C3L-01924.N, C3L-02219.N, C3L-02345.N, C3L-02348.N, C3L-02350.N, C3L-02365.N, C3L-02508.N, C3L-02549.N, C3N-00167.N, C3N-00169.N, C3N-00175.N, C3N-00180.N, C3N-00199.N, C3N-00203.N, C3N-00217.N, C3N-00223.N, C3N-0029

In [133]:
cols = list(del_wt.columns[:-2])

# Get only sig sites
l_sig = u.wrap_ttest(del_wt, 'Mutation', cols, correction_method = 'fdr_bh')
if l_sig is not None:
    l_sig_list = list(l_sig.Comparison)
else: 
    l_sig_list = None
print('significant pvals: \n',l_sig)

# Get all pvals
l_pval = u.wrap_ttest(del_wt, 'Mutation', cols, return_all = True, correction_method = 'fdr_bh')
l_pval = l_pval.rename(columns = {'Comparison': 'Proteomics','P_Value': 'Luad_P_Value'})
#l_pval = l_pval.replace(to_replace = r'_NP_.*$', value = '', regex = True)
l_pval # isoforms for some proteins

significant pvals: 
           Comparison       P_Value
0    EGFR_proteomics  9.121167e-07
1  PIK3CD_proteomics  4.312883e-04
2    XIAP_proteomics  4.264738e-03
3   FOXO1_proteomics  5.642364e-03


,Proteomics,Luad_P_Value
0,EGFR_proteomics,9.121167e-07
1,PIK3CD_proteomics,4.312883e-04
2,XIAP_proteomics,4.264738e-03
3,FOXO1_proteomics,5.642364e-03
4,GRB2_proteomics,7.160789e-03
5,PTEN_proteomics,1.435202e-02
6,AKT1_proteomics,2.191472e-02
7,CCND1_proteomics,2.581832e-02
8,PIK3CB_proteomics,4.551992e-02
9,PIK3CA_proteomics,7.637239e-02


In [ ]:
#Isoform code - fix later
#del_wt.replace(to_replace = r'_NP_.*$', value = '', regex = True)

In [134]:
d = del_wt[del_wt.Mutation == "Deletion"]
wt = del_wt[del_wt.Mutation == "No_Mutation"]
del_med = d.median()
wt_med = wt.median()

l_d = {}

for prot in cols:
    dif = wt_med[prot] - del_med[prot]
    l_d[prot] = dif
    #print(wt_med[prot+'_proteomics'], '-', del_med[prot+'_proteomics'], '=',dif)

l_df = pd.DataFrame.from_dict(l_d, orient='index', columns=['Luad_Median'])
l_df = l_df.reset_index().rename(columns={'index':'Proteomics'})
l_df

,Proteomics,Luad_Median
0,AKT1_proteomics,-0.28010
1,BAD_proteomics,0.05135
2,CCND1_proteomics,1.18450
3,EGFR_proteomics,-2.32970
4,FOXO1_proteomics,0.57005
5,GAB1_proteomics,-0.02360
6,GRB2_proteomics,0.55885
7,GSK3B_proteomics,0.04540
8,INPP4B_proteomics,-1.87510
9,KRAS_proteomics,0.27130


In [135]:
l_merged = l_pval.merge(l_df, on='Proteomics',how='inner')
#l_merged = l_merged.replace(to_replace = r'_proteomics$', value = '', regex = True)
l_merged

,Proteomics,Luad_P_Value,Luad_Median
0,EGFR_proteomics,9.121167e-07,-2.32970
1,PIK3CD_proteomics,4.312883e-04,0.84795
2,XIAP_proteomics,4.264738e-03,-0.52480
3,FOXO1_proteomics,5.642364e-03,0.57005
4,GRB2_proteomics,7.160789e-03,0.55885
5,PTEN_proteomics,1.435202e-02,0.36490
6,AKT1_proteomics,2.191472e-02,-0.28010
7,CCND1_proteomics,2.581832e-02,1.18450
8,PIK3CB_proteomics,4.551992e-02,0.28425
9,PIK3CA_proteomics,7.637239e-02,0.13565


In [136]:
n = g_merged.merge(h_merged, on='Proteomics',how='outer')
n = n.merge(l_merged, on='Proteomics',how='outer')
n= n.merge(b_merged, on='Proteomics',how='outer')
n = n.merge(o_merged, on='Proteomics',how='outer')
n = n.merge(e_merged, on='Proteomics',how='outer')
all_df = n.merge(c_merged, on='Proteomics',how='outer')
all_df



,Proteomics,Gbm_P_Value,Gbm_Median,Hnscc_P_Value,Hnscc_Median,Luad_P_Value,Luad_Median,Brca_P_Value,Brca_Median,Ov_P_Value,Ov_Median,En_P_Value,En_Median,Colon_P_Value,Colon_Median
0,PTEN_proteomics,1.219523e-07,0.511726,0.000023,0.318812,1.435202e-02,0.36490,0.000004,1.00800,0.000909,0.169972,0.000640,0.74660,0.012872,0.20400
1,MAPK3_proteomics,1.020660e-06,0.665105,0.129494,0.061417,5.256187e-01,-0.33845,0.285412,0.22335,0.842428,-0.059507,0.272111,-0.08200,0.003390,-0.19490
2,PIK3CD_proteomics,1.030815e-03,0.492099,0.014016,0.238171,4.312883e-04,0.84795,0.836701,-0.15015,0.937984,0.035308,0.045489,0.18243,0.131200,0.07600
3,GRB2_proteomics,1.089951e-03,0.312146,0.104723,0.190047,7.160789e-03,0.55885,0.061517,0.31470,0.086428,-0.184243,0.160116,0.26400,0.827435,-0.01680
4,PIK3C3_proteomics,1.225949e-03,0.102518,0.312904,0.041287,5.491624e-01,0.10100,0.445080,-0.11490,0.444583,-0.104687,0.036592,0.13160,0.116419,0.07995
5,USP7_proteomics,4.017792e-03,-0.154035,0.064384,-0.094081,3.715141e-01,-0.13260,0.929331,0.14305,0.525069,0.017808,0.572500,-0.03834,0.237441,-0.04450
6,MAP2K1_proteomics,4.138456e-03,0.592389,0.550116,-0.008519,2.426636e-01,0.10475,0.757927,0.03045,0.472124,0.025625,0.675793,0.09190,0.717025,0.02675
7,PIK3R1_proteomics,4.282296e-03,0.188390,0.434622,-0.072064,6.486765e-01,-0.47740,0.164475,1.03615,0.219977,-0.179425,0.135943,-0.33990,0.972777,0.01245
8,MAST2_proteomics,1.260293e-02,0.139061,0.713094,0.081410,9.919357e-01,0.01015,0.213636,0.29705,0.164860,0.085809,0.228439,-0.07090,NaN,NaN
9,INPP4B_proteomics,1.455578e-02,0.341829,0.000676,0.591596,2.766716e-01,-1.87510,0.295970,0.37695,0.353620,-0.370647,0.489186,-0.16320,0.118135,-0.08220


In [152]:
# Keep only genes significant in > 1 cancer
all_sig = []

sig_lists = [g_sig_list, e_sig_list, b_sig_list, o_sig_list, c_sig_list, h_sig_list, l_sig_list]

for next_list in sig_lists:
    if next_list is not None:
        all_sig.append(next_list)
        
flat_list = [item for sublist in all_sig for item in sublist]
sig = set(flat_list) # remove duplicates


bool_df = all_df['Proteomics'].isin(sig)
sig_df = all_df[bool_df]
print(len(sig_df))
sig_df

15


,Proteomics,Gbm_P_Value,Gbm_Median,Hnscc_P_Value,Hnscc_Median,Luad_P_Value,Luad_Median,Brca_P_Value,Brca_Median,Ov_P_Value,Ov_Median,En_P_Value,En_Median,Colon_P_Value,Colon_Median
0,PTEN_proteomics,1.219523e-07,0.511726,0.000023,0.318812,1.435202e-02,0.36490,0.000004,1.00800,0.000909,0.169972,0.000640,0.74660,0.012872,0.20400
1,MAPK3_proteomics,1.020660e-06,0.665105,0.129494,0.061417,5.256187e-01,-0.33845,0.285412,0.22335,0.842428,-0.059507,0.272111,-0.08200,0.003390,-0.19490
2,PIK3CD_proteomics,1.030815e-03,0.492099,0.014016,0.238171,4.312883e-04,0.84795,0.836701,-0.15015,0.937984,0.035308,0.045489,0.18243,0.131200,0.07600
3,GRB2_proteomics,1.089951e-03,0.312146,0.104723,0.190047,7.160789e-03,0.55885,0.061517,0.31470,0.086428,-0.184243,0.160116,0.26400,0.827435,-0.01680
4,PIK3C3_proteomics,1.225949e-03,0.102518,0.312904,0.041287,5.491624e-01,0.10100,0.445080,-0.11490,0.444583,-0.104687,0.036592,0.13160,0.116419,0.07995
5,USP7_proteomics,4.017792e-03,-0.154035,0.064384,-0.094081,3.715141e-01,-0.13260,0.929331,0.14305,0.525069,0.017808,0.572500,-0.03834,0.237441,-0.04450
6,MAP2K1_proteomics,4.138456e-03,0.592389,0.550116,-0.008519,2.426636e-01,0.10475,0.757927,0.03045,0.472124,0.025625,0.675793,0.09190,0.717025,0.02675
7,PIK3R1_proteomics,4.282296e-03,0.188390,0.434622,-0.072064,6.486765e-01,-0.47740,0.164475,1.03615,0.219977,-0.179425,0.135943,-0.33990,0.972777,0.01245
9,INPP4B_proteomics,1.455578e-02,0.341829,0.000676,0.591596,2.766716e-01,-1.87510,0.295970,0.37695,0.353620,-0.370647,0.489186,-0.16320,0.118135,-0.08220
12,EGFR_proteomics,4.010871e-02,-1.001035,0.017719,-0.467227,9.121167e-07,-2.32970,0.016614,-0.58645,0.838960,0.063388,0.006128,-0.52600,0.366886,0.02350


In [153]:
# see sig in cancer
cancer = ['Gbm','Endo','Brca','Ov','Col','Hn','Lung']
i = 0
for next_list in sig_lists:

    print(cancer[i])
    print(next_list, '\n')
    if (i < 6):
        i += 1
    

Gbm
['PTEN_proteomics', 'MAPK3_proteomics', 'PIK3CD_proteomics', 'GRB2_proteomics', 'PIK3C3_proteomics', 'USP7_proteomics', 'MAP2K1_proteomics', 'PIK3R1_proteomics'] 

Endo
['PTEN_proteomics', 'TP53_proteomics', 'PIK3CA_proteomics', 'CSNK2A1_proteomics'] 

Brca
['PTEN_proteomics'] 

Ov
['PTEN_proteomics'] 

Col
None 

Hn
['PTEN_proteomics', 'INPP4B_proteomics', 'PIK3CA_proteomics'] 

Lung
['EGFR_proteomics', 'PIK3CD_proteomics', 'XIAP_proteomics', 'FOXO1_proteomics'] 



In [177]:
def HasPosNeg(row):
    hasPos = False
    hasNeg= False

    for item in row:
        print(item)
        #item = float(item)
        if pd.isnull(item):
            continue
        if item < -0.4:
            hasNeg = True
        if item > 0.4:
            hasPos = True
           
    if hasPos & hasNeg:
        return True
    return False



In [178]:


n = sig_df.iloc[1:2,1:]

HasPosNeg()


Gbm_P_Value


TypeError: '<' not supported between instances of 'str' and 'float'

Combine

In [ ]:
sig_df.to_csv('at_least_one_significant_interacting.csv')

In [ ]:
stdev = ip_df.std()

In [ ]:
stdev = stdev.to_frame()

In [ ]:
stdev.loc[stdev[0] > .2]

In [ ]:
d_stdev = stdev.std()
d_stdev

In [ ]:
a = stdev.hist(bins = 20)
plt.title("Standard Deviations of Pancancer CNV deletions and amplifications of PTEN on Interacting Proteins\n")
plt.xlabel('StDeV')
plt.ylabel('Count')

In [ ]:
plt.rcParams['figure.figsize']=(10,8)
sns.set(font_scale = 1.3)
a = sns.distplot(stdev, bins = 20)

a.set_title("Standard Deviation Distribution for Pancancer PTEN Mutation Effect on Proteomics\n\n (Median of proteomics with Wildtype PTEN - Median of proteomics with Mutated PTEN)")
a.set(xlabel = 'StDeV', ylabel = 'Frequency')

Mean

In [ ]:
mean_df = stdev.mean()

In [ ]:
a = mean_df.hist(bins = 20)
plt.title("Distribution of the Mean between Cancers for PTEN Mutation Effect on all Proteins\n\n (Median of proteomics with Wildtype PTEN - Median of proteomics with Mutated PTEN)")
plt.xlabel('Mean')
plt.ylabel('Count')

In [ ]:
m_df = stdev.median()

In [ ]:
a = m_df.hist(bins = 20)
plt.title("Distribution of the Medians between Cancers for PTEN Mutation Effect on all Proteins\n\n (Median of proteomics with Wildtype PTEN - Median of proteomics with Mutated PTEN)")
plt.xlabel('Median')
plt.ylabel('Count')